In [4]:
gameFilename = '2021games.txt'
teamFilename = '2021teams.txt'
k = 10

import pandas as pd
teamNames = pd.read_csv(teamFilename, header = None)
numTeams = len(teamNames)

games = pd.read_csv(gameFilename, header = None)
numGames = len(games)


import numpy as np

colleyMatrix = 2*np.diag(np.ones(numTeams))
b = np.ones(numTeams)

for i in range(numGames):
    team1ID = games.loc[i, 2] - 1 # subtracting 1 since python indexes at 0
    team1Score = games.loc[i, 4]
    team2ID = games.loc[i, 5] - 1 # subtracting 1 since python indexes at 0
    team2Score = games.loc[i, 7]
    
    colleyMatrix[team1ID, team2ID] -= 1
    colleyMatrix[team2ID, team1ID] -= 1

    colleyMatrix[team1ID, team1ID] += 1
    colleyMatrix[team2ID, team2ID] += 1
    
    if team1Score > team2Score:
        b[team1ID] += 1/2
        b[team2ID] -= 1/2
    elif team1Score < team2Score:
        b[team1ID] -= 1/2
        b[team2ID] += 1/2
    else:  # it is a tie and make 1/2 a win and 1/2 a loss for both teams
        b[team1ID] += 0; # this equates to adding nothing
        b[team2ID] += 0; # clearly this code could be deleted
        
r = np.linalg.solve(colleyMatrix,b)
iSort = np.argsort(-r)

print('\n\n************** COLLEY Rating Method **************\n')
print('===========================')
print('Rank   Rating    Team   ')
print('===========================')
if k==0:
    numberTeamToPrint = numTeams
else:
    numberTeamToPrint = k

for i in range(numberTeamToPrint):
    print(f'{i+1:4d}   {r[iSort[i]]:.5f}  {teamNames.loc[iSort[i],1]}')

print('')   # extra carriage return

numberCorrectPredictions = 0
for i in range(numGames):
    team1ID = games.loc[i, 2] - 1 
    team1Score = games.loc[i, 4]
    team2ID = games.loc[i, 5] - 1 
    team2Score = games.loc[i, 7]
    
    if team1Score > team2Score and r[team1ID] > r[team2ID]:
        numberCorrectPredictions += 1
    elif team2Score > team1Score and r[team2ID] > r[team1ID]:
        numberCorrectPredictions += 1
    elif team1Score == team2Score and r[team1ID] == r[team2ID]:
        numberCorrectPredictions += 1

print(f'Predictability: {numberCorrectPredictions/numGames*100:.2f}%') 




************** COLLEY Rating Method **************

Rank   Rating    Team   
   1   0.71135   Utah
   2   0.69837   Phoenix
   3   0.66527   Philadelphia
   4   0.65228   Brooklyn
   5   0.64642   LA_Clippers
   6   0.64642   Denver
   7   0.62631   Milwaukee
   8   0.58148   Portland
   9   0.58148   Dallas
  10   0.58148   LA_Lakers

Predictability: 65.93%
